# Field Projection with `select()`

This notebook demonstrates how to use FireProx's `.select()` API to project
Firestore documents down to the fields you care about. The examples cover both
synchronous and asynchronous query flows.

## Setup

For the demo we assume that the Firestore emulator is running and that the
`fire_prox.testing` helpers are available to provide a clean database per run.

In [ ]:
from fire_prox.testing import firestore_harness
from fire_prox import FireProx, AsyncFireProx
from google.cloud import firestore
import asyncio

harness = firestore_harness()
harness.__enter__()
client = firestore.Client(project=harness.project_id)
db = FireProx(client)


## Sync: Projecting Specific Fields

When you only need a couple of fields you can ask FireProx to return lightweight
Python dictionaries. Any `DocumentReference` values are automatically converted
back into `FireObject` instances so you can keep navigating relationships.

In [ ]:
users = db.collection('users')
mentor = users.new()
mentor.name = 'Mentor'
mentor.role = 'Staff'
mentor.save(doc_id='mentor')

student = users.new()
student.name = 'Student'
student.mentor = mentor
student.role = 'Learner'
student.save(doc_id='student')

projected = users.select('name', 'mentor').where('role', '==', 'Learner').get()
projected


Each dictionary contains only the requested fields, and references are returned
as fully usable `FireObject` instances:

In [ ]:
student_data = projected[0]
student_data['name'], student_data['mentor'].path


## Streaming Projected Results

The same projection works with `stream()` or `get_all()` for memory-efficient
pipelines.

In [ ]:
for row in users.select(['name']).stream():
    print(row)


## Async Example

Async projections return plain dictionaries as well. Any references are wrapped
in `AsyncFireObject` proxies with lazy-loading intact.

In [ ]:
async def async_demo():
    async_client = firestore.AsyncClient(project=harness.project_id)
    async_db = AsyncFireProx(async_client)
    users = async_db.collection('users_async')

    mentor = users.new()
    mentor.name = 'Async Mentor'
    mentor.role = 'Staff'
    await mentor.save(doc_id='mentor')

    student = users.new()
    student.name = 'Async Student'
    student.mentor = mentor
    student.role = 'Learner'
    await student.save(doc_id='student')

    results = await users.select('name', 'mentor').where('role', '==', 'Learner').get()
    return results

asyncio.run(async_demo())


Don't forget to close the harness when you finish experimenting.

In [ ]:
harness.__exit__(None, None, None)
